In [ ]:
import os
drive_path = 'C:/AIJAVAProject/沛綸小組_2021/model/'
if not os.path.isdir(drive_path):
    os.makedirs(drive_path)

In [ ]:
# encoding: utf-8
import sys
sys.path.append('C:/AIJAVAProject/沛綸小組_2021/local_training')
sys.path.append('C:/AIJAVAProject/沛綸小組_2021/local_training/keras_position_wise_feed_forward')
sys.path.append('C:/AIJAVAProject/沛綸小組_2021/local_training/tensorflow_fast_attention')
sys.path.append('C:/AIJAVAProject/沛綸小組_2021/local_training/keras_performer')
import unittest
import numpy as np
from keras_performer import performer_ver_4 as pfr
from os import listdir
from os.path import isfile, isdir, join
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import DataGeneratorTrain as DGTrain
import DataGeneratorValidation as DGValidation
from matplotlib import pyplot as plt

In [ ]:
def saveDictionary(dt, file):
    import pickle
    a_file = open(file, "wb")
    pickle.dump(dt, a_file)
    a_file.close()

def loadDictionary(file):
    import pickle
    a_file = open(file, "rb")
    dt = pickle.load(a_file)
    return dt

def load(model_name):
    from keras_performer import performer_ver_4 #需要更正是哪一版的performer
    from tensorflow import keras
    from keras_embed_sim import EmbeddingRet, EmbeddingSim
    from keras_pos_embd import TrigPosEmbedding
    from tensorflow_fast_attention.fast_attention import softmax_kernel_transformation , Attention, SelfAttention #新增softmax_kernel_transformation
    from keras_position_wise_feed_forward.feed_forward import FeedForward 

    co = performer_ver_4.get_custom_objects()  #需要更正是哪一版的performer
    co["softmax_kernel_transformation"] = softmax_kernel_transformation #新增softmax_kernel_transformation
    
    model = keras.models.load_model(model_name, custom_objects= co)
    s = loadDictionary(drive_path + 'source_token_dict.pickle')
    t = loadDictionary(drive_path + 'target_token_dict.pickle')
    t_inv = loadDictionary(drive_path + 'target_token_dict_inv.pickle')
        
def loadMaxLen(filename):     
    with open(filename) as f:
        lines = f.readlines()
        return lines
    
def plotTrainingLoss(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['loss', 'val_loss'], loc='upper right') 
    plt.show()
    
def plotTrainingEFFOLoss(history):
    plt.plot(history.history['classifier_decoder_model_loss'])
    plt.plot(history.history['val_classifier_decoder_model_loss'])
    plt.title('model error_feed_forward_output1 loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['loss', 'val_loss'], loc='upper right') 
    plt.show()
    
def plotTrainingDOLoss(history):
    plt.plot(history.history['classifier_decoder_model_1_loss'])
    plt.plot(history.history['val_classifier_decoder_model_1_loss'])
    plt.title('model Decoder-Output loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['loss', 'val_loss'], loc='upper right') 
    plt.show()
    
def plotTrainingEFFOBinAcc(history):
    plt.plot(history.history['classifier_decoder_model_binary_accuracy'])
    plt.plot(history.history['val_classifier_decoder_model_binary_accuracy'])
    plt.title('model error_feed_forward_output1 binary_accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['accuracy', 'val_accuracy'], loc='upper left') 
    plt.show()
    
def plotTrainingDOAcc(history):
    plt.plot(history.history['classifier_decoder_model_1_sparse_categorical_accuracy'])
    plt.plot(history.history['val_classifier_decoder_model_1_sparse_categorical_accuracy'])
    plt.title('model Decoder-Output accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['accuracy', 'val_accuracy'], loc='upper left') 
    plt.show()

In [ ]:
class TestTranslate(unittest.TestCase):

    def test_translate(self):
        source_token_dict = loadDictionary(drive_path + 'source_token_dict.pickle')
        target_token_dict = loadDictionary(drive_path + 'target_token_dict.pickle')
        target_token_dict_inv = loadDictionary(drive_path + 'target_token_dict_inv.pickle')
        
        print("source_token_dict len: ", len(source_token_dict))
        print("target_token_dict len: ", len(target_token_dict))
        print("target_token_dict_inv len: ", len(target_token_dict_inv))
        
        source_max_len_loaded = loadMaxLen(drive_path + "source_max_len.txt")
        source_max_len = int(source_max_len_loaded[0])
        
        target_max_len_loaded = loadMaxLen(drive_path + "target_max_len.txt")
        target_max_len = int(target_max_len_loaded[0])
        
        max_seq_len=source_max_len #(source_max_len, target_max_len)
            
        # Build & fit model      
        model = pfr.get_model(
            max_input_len=(source_max_len, target_max_len), 
            token_num=max(len(source_token_dict), len(target_token_dict)),
            embed_dim=32,
            encoder_num=6,
            decoder_num=6,
            head_num=4,
            hidden_dim=128,
            dropout_rate=0.05,
            use_same_embed=False  # Use different embeddings for different languages
        )

        losses = {
                "classifier_decoder_model": "binary_crossentropy",
                "classifier_decoder_model_1": "sparse_categorical_crossentropy",
        }
        lossWeights = {"classifier_decoder_model": 1.0, "classifier_decoder_model_1": 1.0}
        metrics = {"classifier_decoder_model": tf.keras.metrics.BinaryAccuracy(), "classifier_decoder_model_1": tf.keras.metrics.SparseCategoricalAccuracy()}
        model.compile(optimizer=Adam(learning_rate=0.0001), loss=losses, loss_weights=lossWeights, metrics=metrics)     
        model.summary()
        
        #for output 
        output_buffer_params = { 
            "data_path": ["C:\\AIJAVAProject\\沛綸小組_2021\\model\\", "C:\\AIJAVAProject\\沛綸小組_2021\\model\\"],
            "data_number":[11011, 11011],
            "data_type": [int, int],
            "block_size": [11011, 11011]
            }
        #for input
        input_buffer_params = {
            "data_path": ["C:\\AIJAVAProject\\沛綸小組_2021\\model\\", "C:\\AIJAVAProject\\沛綸小組_2021\\model\\"],
            "data_number":[11011, 11011],
            "data_type": [int, int],
            "block_size": [11011, 11011]
            }
        
        #for output 
        validation_output_buffer_params = { 
            "data_path": ["C:\\AIJAVAProject\\沛綸小組_2021\\model\\", "C:\\AIJAVAProject\\沛綸小組_2021\\model\\"],
            "data_number":[2753, 2753],
            "data_type": [int, int],
            "block_size": [2753, 2753]
            }
        #for input
        validation_input_buffer_params = {
            "data_path": ["C:\\AIJAVAProject\\沛綸小組_2021\\model\\", "C:\\AIJAVAProject\\沛綸小組_2021\\model\\"],
            "data_number":[2753, 2753],
            "data_type": [int, int],
            "block_size": [2753, 2753]
            }
        # Generators
        training_generator = DGTrain.DataGeneratorTrain(input_buffer_params, output_buffer_params,
                                               [list(range(11011)), list(range(11011))],
                                               max_text_len=max_seq_len,
                                              )
        
        validation_generator = DGValidation.DataGeneratorValidation(validation_input_buffer_params, validation_output_buffer_params,
                                               [list(range(2753)), list(range(2753))],
                                               max_text_len=max_seq_len,
                                              )
        
        # Train model on dataset
        EPOCHS = "1000"
        
        model_checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
            filepath=drive_path+"checkpoint_" + EPOCHS + ".h5",
            monitor='val_loss',
            mode='min',
            save_best_only=True)
            
        history = model.fit_generator( 
            generator=training_generator,        
            epochs=int(EPOCHS),    
            verbose=2,
            validation_data=validation_generator,
            callbacks=[model_checkpoint_callback],
        )
        saveDictionary(history.history, drive_path + "history_" + EPOCHS)
        np.save(drive_path + "history_" + EPOCHS + "_npy.npy", history.history)
        model.save(drive_path + "test_model_" + EPOCHS + ".h5")
        
        #print(history.history.keys()) 
        plotTrainingLoss(history)
        plotTrainingEFFOLoss(history)
        plotTrainingDOLoss(history)

        plotTrainingEFFOBinAcc(history)
        plotTrainingDOAcc(history)
        
x=TestTranslate()
x.test_translate()